## What are the features in this data?
* Each instance is a school.

## Graduation rate is given by two features--Cohort and Rate--for each subpopulation below.
* Cohort - Number of students in that subpopulation
* Rate - Percentage (or range of percentage) of students in the cohort graduating with a high school diploma within 4 years

## School identifiers
* STNAM  - State name
* FIPST  - 2 digit code for the state
* LEANM  - School district name
* LEAID  - 7 digit code for school district 
* SCHNAM - School name
* NCESSH - 12 digit school id (only unique identifier for a school)


## Subpopulations
* ALL 	= All students in the school
* MAM 	= American Indian/Alaska   Native students
* MAS 	= Asian/Pacific Islander students
* MHI 	= Hispanic students
* MBL 	= Black students
* MWH 	= White students
* MTR 	= Two or More Races
* CWD 	= Children with Disabilities (IDEA)
* ECD 	= Economically Disadvantaged students
* LEP 	= Limited English Proficient students


In [ ]:
import sys,os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from swampy import structshape as ss
import boto3
import shelve

### Ingest raw data from S3

In [ ]:
def make_raw_gr_frame(year):
    """Create raw graduation rate dataframe for a given year from S3. It is 'raw' because
    no modifications are done and it is cached locally as-is."""
    conn = boto3.client('s3')

    # Graduation rate raw data filenames on S3
    year_to_loc = {2018 : "grad_rate/acgr-sch-sy2018-19-wide.csv",
        2017 : "grad_rate/acgr-sch-sy2017-18.csv",
        2016 : "grad_rate/acgr-sch-sy2016-17.csv", 
        2015 : "grad_rate/acgr-sch-sy2015-16.csv",
        2014 : "grad_rate/acgr-release2-sch-sy2014-15.csv",
        2013 : "grad_rate/acgr-sch-sy2013-14.csv",
        2012 : "grad_rate/acgr-sch-sy2012-13.csv",
        2011 : "grad_rate/acgr-sch-sy2011-12.csv",
        2010 : "grad_rate/acgr-sch-sy2010-11.csv"}
    
    # Verify input parameter is valid
    if year not in list(range(2010, 2019)):
        raise ValueError("input parameter {} is out of range.".format(year))

    # Local storage (cache) for the raw data so that iteration time is faster.
    shelf = shelve.open("gr_dfs")

    df = pd.DataFrame()
    shelf_key = str(year)

    if shelf_key in shelf:
        df = shelf[shelf_key]
    else:
        tmp_file_name = 'curr_file.csv.bak' 
        conn.download_file('edu-data-bucket',year_to_loc[year],tmp_file_name)

        df = pd.read_csv(tmp_file_name)
        shelf[shelf_key] = df

        if os.path.exists(tmp_file_name):
            os.remove(tmp_file_name)

    # Shelve teardown
    shelf.close()
    return df


In [ ]:
def year_string(y: int):
    """Input an integer year and get a range that matches the column suffixes in the raw data.
    e.g. 2011 => 1112 and 2018 => 1819."""
    return str(y)[-2:] + str(int(str(y)[-2:]) + 1)


dfs = [make_raw_gr_frame(year=y) for y in range(2010, 2019)]
years = [year_string(y) for y in range(2010, 2019)]


In [ ]:
# Many of the column names have the school year in them, which presents a challenge for combining all the years
# into one large dataframe.
print(dfs[0].columns)


### Reshaping the data from different years to so that they can be combined into one large dataset. 

In [ ]:
shape_data = [(school_year, df.shape) for school_year, df in zip(years, dfs)]
shape = pd.DataFrame(shape_data, columns=('school_year', 'shape'))
shape


In [ ]:
# Inspect features that are present in some but not common to all.
# Start by removing the years from the column names.
# cols_wo_year => Column names without the year
cols_wo_year = [list(map(lambda x: x.replace(y, ""), df.columns))
                for y, df in zip(years, dfs)]
print(set(cols_wo_year[3]) - set(cols_wo_year[0]))
print(set(cols_wo_year[6]) - set(cols_wo_year[0]))
print(set(cols_wo_year[7]) - set(cols_wo_year[0]))
print(set(cols_wo_year[8]) - set(cols_wo_year[0]))


In [ ]:
# INSERT_DATE refers to when the data was inserted and is not relevant for our study.
dfs[3].drop(['INSERT_DATE'], axis=1, inplace=True)
# ST_SCHID and ST_LEAID are values assigned by the state which are not found in the other years. From the data, it looks like maybe these
# started being assigned in 2016. If we need another geographical grouping mechanism in the future we can look into it.
dfs[6].drop(['ST_LEAID', 'ST_SCHID'], axis=1, inplace=True)
# HOM_COHORT and FCS_COHORT refer to the subpopulation of homeless and foster care students, which was not tracked before school year 2017-2018
idx7_sr = shape.school_year[7]
idx8_sr = shape.school_year[8]
dfs[7].drop(['ST_LEAID', 'ST_SCHID', 'FCS_RATE_'+idx7_sr, 'FCS_COHORT_'+idx7_sr,
            'HOM_RATE_'+idx7_sr, 'HOM_COHORT_'+idx7_sr], axis=1, inplace=True)
dfs[8].drop(['ST_LEAID', 'ST_SCHID', 'FCS_RATE_'+idx8_sr, 'FCS_COHORT_'+idx8_sr,
            'HOM_RATE_'+idx8_sr, 'HOM_COHORT_'+idx8_sr], axis=1, inplace=True)


### Verify that all dataframes have the same columns before we combine.

In [ ]:
cols_wo_school_year = [list(map(lambda x: x.replace(y, ""), df.columns))
                       for y, df in zip(years, dfs)]
for num1, num2 in zip(range(0, 8), range(1, 9)):
    assert cols_wo_school_year[num1] == cols_wo_school_year[num2]


In [ ]:
big_df = pd.DataFrame()
print("big_df_columns", big_df.columns)
for idx, df in enumerate(dfs):
    df.columns = cols_wo_school_year[0]
    df['Year'] = list(range(2010,2019))[idx]
    # reorder columns to be how we want
    df = df[['Year']+cols_wo_school_year[0]]
    big_df = pd.concat([big_df, df], axis=0)


In [ ]:
big_df.shape, big_df.head(n=3)


### How are missing values distributed?

In [ ]:
df_info = pd.DataFrame(columns=['% Null', '% Unique'])
df_info['% Null'] = big_df.isnull().sum() / len(big_df) * 100
df_info['% Unique'] = big_df.nunique() / len(big_df) * 100
df_info


In [ ]:
plt.plot(range(big_df.columns.size),
         (big_df.isnull().sum() / len(big_df) * 100).values)
plt.xticks(ticks=list(range(big_df.columns.size)),
           labels=big_df.columns.tolist(), rotation=90)
plt.ylabel('% Null')
# MAMs, MAS, LEP, and MTR are the subpopulations with the largest number of missing values.

### The missing data in the subpopulation columns presents a problem for performing machine learning on this data.
* The Adjusted Cohort Graduation Rate codebook gives the following Q&A:
#### Why doesn’t the summation of the major racial and ethnic groups equal the “ALL” student count?
* Due to flexibilities with states’ implementation of the Elementary and Secondary Education Act,
  there may be instances where not all possible groupings of racial/ethnic identification are reported
  as individual subgroups. Therefore, some information may be missing and these counts by major racial
  and ethnic group will not include every student; however any students not included within an individual
  major racial and ethnic group would be included in the “ALL” student count.
#### Why are the major racial and ethnic groups reported differently by states?
* Under the ESEA, a State educational agency (SEA) has the flexibility to determine the major racial/ethnic
  groups it will use for reporting on the data included in its assessment and accountability system.  The
  subgroups that an SEA uses are approved through its Accountability Workbook (the most recent copy of each
  state’s workbook can be found here:  http://www2.ed.gov/admins/lead/account/stateplans03/index.html).
  As a result, there is some variation in how SEAs report data by race and ethnicity. 
### As a first step, we are going to look at prediction models which do not include minority groupings of the students.

In [ ]:
# Drop population subgroup columns
cols_to_keep = ['Year', 'STNAM', 'FIPST', 'LEAID', 'LEANM',
                'NCESSCH', 'SCHNAM', 'ALL_COHORT_', 'ALL_RATE_']
big_df = big_df[cols_to_keep]


# Feature pre-processing
* Year, ALL_COHORT_, and ALL_RATE_ are numeric columns which are strings (objects) in the raw data.

In [ ]:
# Several columns are of object type, which means that either text or Nan values are present.
def print_object_column_info(df):
    """
    Print column data types and the number of object columns.
    """
    print(big_df.dtypes, "{} object columns present". format(
        len(big_df.select_dtypes(include=object).count())))


print_object_column_info(big_df)


In [ ]:
big_df.select_dtypes(include=object)


In [ ]:
# Convert year feature to numeric
big_df.loc[:, 'Year'] = np.int64(big_df.Year)
print_object_column_info(big_df)


Observation: We notice that their are periods in the ALL_COHORT_ columns and numbers encoded as strings like "GE95" in the ALL_RATE_ column.

In [ ]:
big_df.ALL_COHORT_.describe(), big_df.ALL_RATE_.describe()


In [ ]:
def gather_nonconvertible_items(t: pd.Series):
    """
    Print items in a series that cannot be trivially converted to a number. 
    E.g. '120' can be converted, but '.' or 'LE50' cannot.
    """

    from collections import Counter
    ret = []
    for i in t:
        try:
            tmp = np.int64(i)
        except:
            ret += [i]
    return Counter(ret)


In [ ]:
gather_nonconvertible_items(
    big_df.ALL_COHORT_), gather_nonconvertible_items(big_df.ALL_RATE_)


### Number-like string entry observations
* The ALL_COHORT_ column has 2913 rows with a '.'
* THe ALL_RATE_ column has numbers given as ranges, numbers with GE,LE,LT,GT prefixes, and also 'PS'.
* The codebook explains that these are to conceal specifics when it would give us the ability to identify individual students. The 'PS' specifically means that the cohort had 5 or fewer students in it.

### Decisions
* Drop the ALL_COHORT_ rows because it is only 1.4% of the data. We later decide to estimate these numbers from the Directory dataset, which states the size of the schools.
* Drop the ALL_RATE_ rows that have 'PS' and convert the rest to the average meaning of the term. E.g. GT50 => 75, LT50 => 25, 88-92 => 90.

In [ ]:
# Print out number of schools in each year with 5 or fewer students in cohort.
for df in dfs: 
    [print(sum(df[col] == 'PS')) for col in df.columns if col.startswith('ALL_RATE')]
            

In [ ]:
# Drop rows where ALL_COHORT_ == '.'
instances_after_drop = len(big_df) - 2913
# Reindex the dataframe because their are duplicate indexes
big_df.index = list(range(len(big_df)))
big_df.drop(big_df[big_df.ALL_COHORT_ == '.'].index, axis=0, inplace=True)
assert len(big_df) == instances_after_drop
print("Number of instances after this drop", len(big_df))
# Can convert ALL_COHORT_ column to numeric now
big_df.loc[:,'ALL_COHORT_'] = np.int64(big_df.ALL_COHORT_.values)
big_df.dtypes

In [ ]:
# Drop rows where ALL_RATE_ == 'PS'
print("Dropping", len(big_df[big_df.ALL_RATE_ == 'PS']))
big_df.drop(big_df[big_df.ALL_RATE_ == 'PS'].index, axis=0, inplace=True)
print("Number of instances after this drop", len(big_df))

In [ ]:
# Helper functions to convert ranges given in ALL_RATE_ column to a trivially convertible string 
def conv_range_to_numeric_string(t: str):
    """Take in a numeric range given as a string, e.g. "10-20" and return the midpoint as a string."""
    vals = t.split("-")
    if len(vals) == 2:
        val1, val2 = float(vals[0]), float(vals[1])
        new_val = val1 / 2.0 + val2 / 2.0
        return str(new_val)
    else:
        # If a range with a dash in it is not found, return original string unchanged.
        return t

assert conv_range_to_numeric_string('95-95') == '95.0'
assert conv_range_to_numeric_string('105-110') == '107.5'
assert conv_range_to_numeric_string('90') == '90'

def strip_prefixed_string(t: str):
    """
    Convert prefixed numbers to a string of a number in the midpoint of their range 
    """
    if t.startswith('GT'):
        t = t.removeprefix('GT')
        t = float(t)
        t = sum([t, 100.0])/2.0
        return str(t)
    elif t.startswith('GE'):
        t = t.removeprefix('GE')
        t = float(t)
        t = sum([t, 100.0])/2.0
        return str(t)
    elif t.startswith('LT'):
        t = t.removeprefix('LT')
        t = float(t)
        t = sum([t, 0])/2.0
        return str(t)
    elif t.startswith('LE'):
        t = t.removeprefix('LE')
        t = float(t)
        t = sum([t, 0])/2.0
        return str(t)
    # If one of the prefixes is not found, return original string unchanged
    return t

assert strip_prefixed_string('GE50') == '75.0'
assert strip_prefixed_string('GT50') == '75.0'
assert strip_prefixed_string('LE90') == '45.0'
assert strip_prefixed_string('LT90') == '45.0'
assert strip_prefixed_string('95-95') == '95-95'

In [ ]:
big_df.ALL_RATE_

In [ ]:
# Run the above functions on the series to produce a float64 series
big_df.loc[:, 'ALL_RATE_'] = big_df.ALL_RATE_.map(strip_prefixed_string, na_action='ignore')
big_df.loc[:, 'ALL_RATE_'] = big_df.ALL_RATE_.map(conv_range_to_numeric_string, na_action='ignore')
big_df.loc[:, 'ALL_RATE_'] = pd.to_numeric(big_df.ALL_RATE_.tolist()) 

In [ ]:
big_df.ALL_RATE_

In [ ]:
big_df.dtypes

### Explore distributions, correlations, outliers of numeric variables

In [ ]:
sns.pairplot(big_df)

### Observations
* All graduation rates varies from 0 to 100, which matches expectation
* Schools are not evenly distributed across states. 
* Outlier 1 ALL_COHORT_: For two schools 2017 and 2018, there are class size outliers of > 10000 students all others < 5000 students.
* Outlier 2 LEAID and NCESSH: For the last two years, there are new school district (LEAID) and school id (NCESSCH) outside the range of the others. 
* Outlier 3 FIPST: There is a state code higher than 60, with all of its graduation rates higher than 20%.
* Outlier 4 FIPST: There is one state code, higher than 50, only seen in the last two years of data.
* Outlier 5 LEAID: There is one school district with all graduation rations >= 25%. 

### Outlier 1 - large cohort sizes

In [ ]:
large_school_indexes = big_df.ALL_COHORT_ > 5000
big_df.loc[large_school_indexes,:]

Decision: Based on google search, the graduating classes are closer to 100-200 than the extremely large sizes reported.
We drop these schools since they are called into question. Note that we could impute them with the mean school size.

In [ ]:
big_df.drop(big_df[large_school_indexes].index,axis=0,inplace=True)
big_df.ALL_COHORT_.describe()

### Outliers 2, 3, 4, 5  - new school identifers in 2017 and 2018

In [ ]:
big_df.query('FIPST == 72 and NCESSCH > 6E11 and Year > 2016')

Observation: New Puerto Rico school additions in 2017, either newly created or newly included in the data gathering.

In [ ]:
# 405 Puerto Rico instances added for 2017 and 2018 that have graduation rates between 25 and 97%.
all_puerto_rico_schools = big_df[big_df.STNAM == 'PUERTO RICO']
puerto_rico_schools = big_df[big_df.LEAID > 7E6]
assert(len(all_puerto_rico_schools) == len(puerto_rico_schools))
puerto_rico_schools.Year.describe(), puerto_rico_schools.ALL_RATE_.describe()

### Create a new pairplot now that the outliers have been investigated.

In [ ]:
sns.pairplot(big_df)

Observation: There is a group of large schools in the 3900-5000 cohort size range that are in the same state and school district. This outlier is the Electronic Classroom Of Tomorrow in Ohio. It has a below average graduation rate.

In [ ]:
big_df.query('ALL_COHORT_ > 3200').ALL_RATE_.describe()

### Summary statistics for cohort size and graduation rate 

In [ ]:
big_df.select_dtypes(include=np.number)[['ALL_COHORT_','ALL_RATE_']].describe()
f, (ax, bx) = plt.subplots(2)
plt.subplot(211)
plt.title('Number of students in cohort')
sns.boxplot(big_df.ALL_COHORT_,orient='horiz')
plt.subplot(212)
plt.title('Percentage that graduated')
sns.boxplot(big_df.ALL_RATE_,orient='horiz')
plt.tight_layout(pad=2)
big_df[['ALL_COHORT_','ALL_RATE_']].describe()

### Which schools have < 15% graduation rate?

In [ ]:
under_25_percent_graduation = big_df.query('ALL_RATE_ < 25').NCESSCH.unique()
print(len(under_25_percent_graduation), "unique schools with less than 25 percent graduation rate between 2010-2018")
print("They are spread across", big_df.query('NCESSCH in @under_25_percent_graduation').STNAM.unique().size, "states")

Low performing schools are found distributed across 47 states in the nation.

### Check whether there are graduation rate patterns by state or school district 

In [ ]:
big_df.plot.scatter(x='LEAID',y='ALL_RATE_')

In [ ]:
big_df.plot.scatter(x='FIPST',y='ALL_RATE_')

There are right-shifted graduation rate distributions at state codes around 15,31,38,50,72

### Categorical column analysis
* Check whether state names map one-to-one with FIPST 
* Determine which features uniquely identify a school and which do not

In [ ]:
def create_fipst_to_stnam_dict(df):
    """
    Verify that state codes match state names in a one-to-one fashion. 
    Return dictionary mapping FIPST to STNAM.
    """
    fipst_to_name = {}
    for fipst, name in zip(df.FIPST,df.STNAM):
        if fipst in fipst_to_name:
            try:
                assert name == fipst_to_name[fipst]
            except:
                print("found name {} for fipst {} but {} was in dictionary already".format(name,fipst,fipst_to_name[fipst]))
        else:
            fipst_to_name[fipst] = name
    return fipst_to_name

_ = create_fipst_to_stnam_dict(big_df)

Obversation: Bureau of Indian Affairs was renamed to Bureau of Indian Education in 2006 and some schools were still using the old name.

In [ ]:
# Rename BUREAU OF INDIAN AFFAIRS to BUREAU OF INDIAN EDUCATION
indian_affairs_rows = big_df.query("STNAM.str.contains('INDIAN AFFAIRS')").index
big_df.loc[indian_affairs_rows,'STNAM'] = 'BUREAU OF INDIAN EDUCATION' 

### Create a dictionary mapping state code to state name for later

In [ ]:
fipst_to_name = create_fipst_to_stnam_dict(big_df)
print(len(fipst_to_name.keys()))
sorted(fipst_to_name.items())

### Check whether state and school name are unique 

In [ ]:
# The state and local school names are not unique. Any value over 9, which is the number of years studied, 
# indicates there were two schools with the same name for at least one year.
mask_non_unique_names = big_df[['STNAM','SCHNAM']].value_counts().values > 9
big_df[['STNAM','SCHNAM']].value_counts()[mask_non_unique_names]


Observation: We have to use the NCESSCH to identify schools since it is the only unique identifier.

### Save multi-year graduation rate dataframe to file for future merging.

In [ ]:
big_df.info()

In [ ]:
big_df.to_csv('cleaned_mega_grad_rate_frame.csv.bak',index=False)